<img src=https://audiovisuales.icesi.edu.co/assets/custom/images/ICESI_logo_prin_descriptor_RGB_POSITIVO_0924.jpg width=200>

----

# Examen Parcial Práctico: Regularización, Regresión y Clasificación

_Primer Semestre_

En este examen práctico se evaluarán los siguientes conceptos:
- Regularización Lasso y Ridge en modelos de regresión y clasificación.
- El parámetro C (inverso de la fuerza de regularización).
- Regresión logística y la interpretación de sus coeficientes.
- Técnicas de validación cruzada (por ejemplo, stratified k-folds).
- Métricas de clasificación: F1-score, matriz de confusión, classification report, accuracy, recall.

El dataset a utilizar es el **Forest Covertypes** de scikit-learn.  
Para más información consulta:  
[Forest Covertypes dataset](https://scikit-learn.org/stable/datasets/real_world.html#forest-covertypes)  
[fetch_covtype documentation](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn.datasets.fetch_covtype)


## Instrucciones

1. **Carga y Preprocesamiento:**  
   - Cargar el dataset Forest Covertypes utilizando `sklearn.datasets.fetch_covtype`.
   - Realizar un análisis exploratorio y dividir el dataset en conjuntos de entrenamiento y prueba.
   - (Opcional) Estandarizar los predictores, ya que las técnicas de regularización son sensibles a la escala.

2. **Modelos de Regresión Regularizada:**  
   - Implementa modelos de regresión Ridge y Lasso sobre un problema de regresión (puedes transformar el problema en uno binario o utilizar un subconjunto).
   - Analiza el efecto del parámetro de regularización λ (alpha) sobre la magnitud de los coeficientes.

3. **Modelo de Clasificación:**  
   - Implementa un modelo de regresión logística regularizada (puedes utilizar L1 o L2) para clasificar el dataset original.
   - Interpreta los coeficientes obtenidos, considerando la relación entre los predictores y la variable respuesta.

4. **Validación Cruzada:**  
   - Utiliza Stratified K-Folds (por ejemplo, 5 folds) para evaluar el desempeño del modelo de clasificación.
   - Calcula las métricas (accuracy, recall, F1-score, matriz de confusión y classification report) tanto en validación cruzada como en el conjunto de prueba.

5. **Comentarios y Explicaciones:**  
   - Cada bloque de código debe incluir comentarios explicativos y, cuando sea necesario, fórmulas en los bloques Markdown por ejemplo, la fórmula del F1-score:  
     $$
     F1 = \frac{2 \times \text{precision} \times \text{recall}}{\text{precision} + \text{recall}}
     $$.

----------

A continuación, se muestra el código base del examen:


In [13]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, classification_report

# Cargar el dataset Forest Covertypes

In [14]:
data = fetch_covtype()
X = data.data
y = data.target

## Paso 1: Análisis Exploratorio y Preprocesamiento

Realice un análisis exploratorio riguroso del dataset. Se recomienda:

- Convertir el dataset a un `DataFrame` de pandas y visualizar las primeras filas usando `.head()`.
- Revisar las dimensiones del DataFrame (número de filas y columnas) con `.shape`.
- Verificar la tipología de las variables, determinando cuántas son numéricas y cuántas son categóricas.
- Comprobar si existen valores nulos en cada columna.
- Calcular y visualizar la matriz de correlación de Pearson para las variables numéricas.
- Realizar un análisis univariado mediante histogramas, boxplots y scatterplots (comparando cada variable numérica con la variable objetivo).
- Analizar la distribución de la variable objetivo mediante `value_counts()` y gráficos, para detectar posibles desbalances en el dataset.
- Aplicar técnicas de normalización/estandarización (por ejemplo, Z-Score con StandardScaler y Min-Max Scaling) sobre los predictores.

Este análisis exploratorio (EDA) es fundamental para entender la estructura y calidad del dataset antes de proceder con el modelado.


# Preprocesamiento

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Convertir el dataset a DataFrame

In [19]:
df = pd.DataFrame(X, columns=data.feature_names if hasattr(data, 'feature_names') 
                else [f"Feature_{i}" for i in range(X.shape[1])])
df['target'] = y

# Visualizar las primeras filas y dimensiones del DataFrame

In [8]:
#TU CODIGO VA AQUI

Primeras filas del DataFrame:


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,target
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [7]:
# VERFICIDAR DIMENSIONES DEL DATAFRAME
#TU CODIGO VA AQUI




Dimensiones del DataFrame: (581012, 55)


# Revisar la tipología de variables

In [21]:
#TU CODIGO VA AQUI


# Verificar valores nulos en el DataFrame


In [22]:
#TU CODIGO VA AQUI


# Calcular y visualizar la matriz de correlación de Pearson para las variables numéricas

In [23]:
corr_matrix = df[num_vars].corr(method='pearson')
plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix, cmap='coolwarm')
plt.title("Matriz de Correlación de Pearson")
plt.show()

# Que podemos decir de esta matriz de correlación?

# Análisis bivariado respecto a la variable objetivo

verifica la distribución de las clases

Es un problema balanceado?

In [26]:
#TU CODIGO VA AQUI


# Dividir el DataFrame en conjuntos de entrenamiento y prueba (70%-30%) manteniendo la estratificación

In [11]:
#TU CODIGO VA AQUI


Dimensiones del conjunto de entrenamiento: (406708, 54)
Dimensiones del conjunto de prueba: (174304, 54)


# Aplicar normalización: StandardScaler (Z-Score) y Min-Max Scaling

Ajustar y transformar el conjunto de entrenamiento

In [13]:
scaler_std = StandardScaler()
scaler_mm = MinMaxScaler()
#TU CODIGO VA AQUI




# Transformar el conjunto de prueba

In [14]:
#TU CODIGO VA AQUI



## Paso 2: Modelos de Regresión Regularizada

Ahora que se ha realizado un análisis exploratorio completo y se han generado conjuntos de entrenamiento y prueba con datos escalados, en este paso implementaremos modelos de regresión regularizada (Ridge y Lasso) sobre un problema simplificado.

**Objetivo:**  
- Seleccionar un subconjunto de predictores (por ejemplo, las primeras 5 columnas) y transformar la variable objetivo en un problema binario.  
- Utilizar los datos escalados (por ejemplo, el conjunto obtenido con StandardScaler) para ajustar modelos Ridge y Lasso.  
- Comparar el efecto del parámetro de regularización (alpha) sobre los coeficientes.

*Pistas:*  
- Transforme la variable target a binario, por ejemplo: asignar 1 si el valor es mayor que un umbral (p. ej., 2) y 0 en caso contrario.  
- Observe cómo varían los coeficientes al modificar el parámetro alpha.
  
_Función objetivo de Ridge:_  
$$
\text{Objetivo: } RSS + \lambda \sum_{j=1}^{p} \beta_j^2
$$
_Función objetivo de Lasso:_  
$$
\text{Objetivo: } RSS + \lambda \sum_{j=1}^{p} |\beta_j|
$$


In [25]:
# Seleccionar las primeras 5 variables del conjunto escalado y transformar la variable target
#debes convertir la variable objetivo a una variable binaria
#si es mayor que 2        => 1
# si es menor o igual a 2 = > 0
#TU CODIGO VA AQUI


# Modelo Ridge

In [17]:
ridge_model = Ridge(alpha=1.0)  # Ajuste con alpha = 1.0 (puede variar este parámetro)
#TU CODIGO VA AQUI

#tome nora de sus observaciones

Coeficientes Ridge: [-0.14749998  0.01096182  0.03756682 -0.00143453  0.03491748]


# Modelo Lasso

In [18]:
lasso_model = Lasso(alpha=0.1)  # Ajuste con alpha = 0.1 (puede variar este parámetro)
#TU CODIGO VA AQUI

#tome nora de sus observaciones

Coeficientes Lasso: [-0.05356982  0.          0.         -0.          0.        ]


## Paso 3: Modelo de Clasificación - Regresión Logística Regularizada

En este paso se construirá un modelo de clasificación utilizando regresión logística regularizada. Aproveche los datos escalados obtenidos en el Paso 1.

**Objetivo:**  
- Ajustar un modelo de regresión logística (utilizando penalización L2 o L1) con el conjunto de entrenamiento completo.  
- Interpretar los coeficientes obtenidos, considerando que el parámetro C (inverso de la fuerza de regularización) controla la penalización: valores pequeños de C implican penalización fuerte, mientras que valores mayores permiten mayor flexibilidad.

*Pista:*  
- Use la versión estandarizada del conjunto de entrenamiento (X_train_std) para asegurar que la regularización actúe de manera equitativa en todos los predictores.


In [ ]:
log_reg = LogisticRegression(penalty='l2', C=1.0, solver='liblinear', random_state=42)
#TU CODIGO VA AQUI


#tome nora de sus observaciones

## Paso 4: Validación Cruzada

Utilice la técnica de validación cruzada con Stratified K-Folds para evaluar el desempeño del modelo de clasificación ajustado.

**Objetivo:**  
- Configurar una validación cruzada con 5 folds, que preserve la distribución de la variable objetivo.  
- Obtener predicciones a través de `cross_val_predict` y calcular las métricas: accuracy, recall, F1-score, matriz de confusión y classification report.

*Pista:*  
- Asegúrese de utilizar el conjunto de entrenamiento estandarizado (X_train_std) y la variable objetivo (y_train).

# Validación Cruzada con StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
log_reg = LogisticRegression(penalty='l2', C=1.0, solver='liblinear', random_state=42)
#TU CODIGO VA AQUI



#CUAL ES EL RENDIMIENTO ESPERADO EN EL CONJUNTO DE PRUEBA??


## Paso 5: Evaluación del Modelo en el Conjunto de Prueba

Utilice el modelo de regresión logística entrenado en el Paso 3 para verificar su estimación y realizar predicciones sobre el conjunto de prueba escalado.

**Objetivo:**  
- Calcular y visualizar las métricas de evaluación: accuracy, recall (macro), F1-score (macro), matriz de confusión y classification report.
- Compare los resultados obtenidos en el conjunto de prueba con los de la validación cruzada.

*Pista:*  
- Emplee las funciones `accuracy_score`, `recall_score`, `f1_score`, `confusion_matrix` y `classification_report` de scikit-learn.

In [ ]:
#TU CODIGO VA AQUI


# Evaluación en el conjunto de prueba

# Conclusiones

Agrega las conclusiones más relevantes del análisis realizado y los resultados obtenidos.

- Conclusión 1
- Conclusión 2
- Conclusión 3
